In [5]:
import sys
import io
import os
import decimal
import pandas as pd
import folium

In [6]:
# Read lines from input text file
input_file = open('coordinates.txt', 'r')
sys.stdin = io.StringIO(input_file.read())

lines = sys.stdin.readlines()
lines
input_file.close()

In [7]:
# Global variables
stripLines=[]
preValidLines=[]
invalidLines=[]
validLines=[]

In [8]:
coords = pd.DataFrame(columns=['lat', 'lon', 'label'])

In [9]:
# filter blank lines and comment lines which is starting with (#)

for line in lines:
    l = line.strip()
    isCorrect = 1
    if l != '' and l[0] != '#' and len(l)>3: # filter blank lines and comment lines
        stripLines.append(l)  
        

In [10]:
# find valid lines from prevalid lines for (,)
for pl in stripLines:
    pos = pl.find(',')
    
    # check comma (,) in coordinates 
    if pos == -1 or pos == 0 or pl[len(pl)-1] == ',' or pl.count(',')>1 or pl.count(',')==0:
        invalidLines.append(pl)
    else:
        preValidLines.append(pl)
        

In [11]:
preValids=preValidLines.copy()

In [12]:
# filtering unwanted string from lines
for pl in preValids:

    ll = pl.split(',')
    s = ll[0].strip()
    alphaCount = 0
    for i in range(len(s)):
        if int(ord(s[i]))>=63:
            alphaCount = alphaCount + 1
            
    if alphaCount>1:
        invalidLines.append(pl)
        preValidLines.remove(pl)


In [13]:
preValids=preValidLines.copy()

In [14]:
# filtering unwanted character in coordinates
for pl in preValids:
    ll = pl.split(',')
    s = ll[0].strip()
    lt = s.split(' ')
                
    s = ll[1].strip()
    ln = s.split(' ')

    try:
        lat = float(lt[0].strip())
        lon = float(ln[0].strip())
        
    except ValueError:
        invalidLines.append(pl)
        preValidLines.remove(pl)
        print("Not a float")
    
#     break

Not a float


In [15]:
preValids=preValidLines.copy()

In [16]:
# find single lat lon and validate
for pl in preValids:
    ll = pl.split(',')
    
    s = ll[0].strip()
    lt = s.split(' ')
    
    s = ll[1].strip()
    ln = s.split(' ')
    
    # remove large values from EW
    if decimal.Decimal(lt[0])>180 or decimal.Decimal(lt[0])<-180 or decimal.Decimal(ln[0])>180 or decimal.Decimal(ln[0])<-180: 
        invalidLines.append(pl)
        preValidLines.remove(pl)
        continue
    
    # filter 6 or more decimal places
    if decimal.Decimal(lt[0]).as_tuple().exponent<-5 or decimal.Decimal(ln[0]).as_tuple().exponent<-5: 
        invalidLines.append(pl)
        preValidLines.remove(pl)
        continue
        
    # remove negative values for NEWS
    if len(lt)>1 and len(lt[1]) == 1 and decimal.Decimal(lt[0])<0:
        invalidLines.append(pl)
        preValidLines.remove(pl)
        continue
    if len(ln)>1 and len(ln[1]) == 1 and decimal.Decimal(ln[0])<0:
        invalidLines.append(pl)
        preValidLines.remove(pl)
        continue  
    
    # find valid lines if the line has only decimal values with no label
    if len(lt) == 1 and len(ln) == 1:
        if decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90 or decimal.Decimal(ln[0])>180 or decimal.Decimal(lt[0])<-180:
            invalidLines.append(pl)
            preValidLines.remove(pl)
        else:
            validLines.append(pl)
            preValidLines.remove(pl)
            coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': ''}, ignore_index=True)
            
    # find valid lines where N/S/W/E is added with first decimal and second decimal is single    
    elif len(lt) == 2 and len(ln) == 1:
        # remove large values from NS
        if (lt[1]=='N' or lt[1]=='S') and (decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove SW value if out of range
        elif (lt[1]=='E' or lt[1]=='W') and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90):
            invalidLines.append(pl)
            preValidLines.remove(pl)
        else:
            validLines.append(pl)
            preValidLines.remove(pl)
            if lt[1] == 'N' or lt[1] == 'S':
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': ''}, ignore_index=True)
            else:
                coords = coords.append({'lat': decimal.Decimal(ln[0]), 'lon': decimal.Decimal(lt[0]), 'label': ''}, ignore_index=True)
    
    # find valid lines where first decimal is single and N/S/W/E is added with second decimal      
    elif len(lt) == 1 and len(ln) == 2:
        # remove large values from NS
        if len(ln[1])==1 and (ln[1]=='N' or ln[1]=='S') and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove SW value if out of range
        elif len(ln[1])==1 and (ln[1]=='E' or ln[1]=='W') and (decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90):
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove if there is a label but latitude is out of range
        elif len(ln[1])>1 and (decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        else:
            validLines.append(pl)
            preValidLines.remove(pl)
            if len(ln[1])==1 and (ln[1] == 'N' or ln[1] == 'S'):
                coords = coords.append({'lat': decimal.Decimal(ln[0]), 'lon': decimal.Decimal(lt[0]), 'label': ''}, ignore_index=True)
            elif len(ln[1])==1 and (ln[1] == 'E' or ln[1] == 'W'):
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': ''}, ignore_index=True)
            else:
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': ln[1]}, ignore_index=True)
    
    # find valid lines where first decimal is single and N/S/W/E or a label is added with second decimal       
    elif len(lt) == 1 and len(ln) > 2:
        # remove large values from NS
        if (ln[1]=='N' or ln[1]=='S') and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove if there is a label and longitude is > 90 or <-90 but latitude is out of range
        elif len(ln[1])>1 and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90) and (lt[1]=='E' or lt[1]=='W' or decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        else:
            validLines.append(pl)
            preValidLines.remove(pl)
            label=''
            if len(ln[1])==1:
                for j in range(2, len(ln)):
                    label += ln[j] +' '
            else:
                for j in range(1, len(ln)):
                    label += ln[j] +' '
                    
            if len(ln[1])==1 and (ln[1] == 'N' or ln[1] == 'S'):
                coords = coords.append({'lat': decimal.Decimal(ln[0]), 'lon': decimal.Decimal(lt[0]), 'label': label}, ignore_index=True)
            elif len(ln[1])==1 and (ln[1] == 'E' or ln[1] == 'W'):
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': label}, ignore_index=True)
            else:
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': label}, ignore_index=True)
    
    # find valid lines where first decimal has N/S/W/E and N/S/W/E or a label is added with second decimal  
    elif len(lt) == 2 and len(ln) == 2:
        # remove invalid axis
        if (lt[1]=='N' and ln[1]=='N') or (lt[1]=='S' and ln[1]=='S') or (lt[1]=='E' and ln[1]=='E') or (lt[1]=='W' and ln[1]=='W') or (lt[1]=='N' and ln[1]=='S') or (lt[1]=='S' and ln[1]=='N') or (lt[1]=='E' and ln[1]=='W') or (lt[1]=='W' and ln[1]=='E'):
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove large values from NS
        elif (lt[1]=='N' or lt[1]=='S') and (decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
       # remove large values from NS
        elif len(ln[1])==1 and (ln[1]=='N' or ln[1]=='S') and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove if there is a label and longitude is > 90 or <-90 but latitude is out of range
        elif len(ln[1])>1 and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90) and (lt[1]=='E' or lt[1]=='W' or decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        else:
            validLines.append(pl)
            preValidLines.remove(pl)
            if len(ln[1])==1 and (ln[1] == 'N' or ln[1] == 'S'):
                coords = coords.append({'lat': decimal.Decimal(ln[0]), 'lon': decimal.Decimal(lt[0]), 'label': ''}, ignore_index=True)
            elif len(ln[1])==1 and (ln[1] == 'E' or ln[1] == 'W'):
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': ''}, ignore_index=True)
            else:
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': ln[1]}, ignore_index=True)
            
    # find valid lines where first decimal has N/S/W/E and second decimal has N/S/W/E and a label 
    elif len(lt) == 2 and len(ln) > 2:
        # remove invalid axis
        if (lt[1]=='N' and ln[1]=='N') or (lt[1]=='S' and ln[1]=='S') or (lt[1]=='E' and ln[1]=='E') or (lt[1]=='W' and ln[1]=='W') or (lt[1]=='N' and ln[1]=='S') or (lt[1]=='S' and ln[1]=='N') or (lt[1]=='E' and ln[1]=='W') or (lt[1]=='W' and ln[1]=='E'):
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove large values from NS
        if (ln[1]=='N' or ln[1]=='S') and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        # remove if there is a label and longitude is > 90 or <-90 but latitude is out of range
        elif len(ln[1])>1 and (decimal.Decimal(ln[0])>90 or decimal.Decimal(ln[0])<-90) and (lt[1]=='E' or lt[1]=='W' or decimal.Decimal(lt[0])>90 or decimal.Decimal(lt[0])<-90): 
            invalidLines.append(pl)
            preValidLines.remove(pl)
        else:
            validLines.append(pl)
            preValidLines.remove(pl)
            label=''
            if len(ln[1])==1:
                for j in range(2, len(ln)):
                    label += ln[j] +' '
            else:
                for j in range(1, len(ln)):
                    label += ln[j] +' '
                    
            if len(ln[1])==1 and (ln[1] == 'N' or ln[1] == 'S'):
                coords = coords.append({'lat': decimal.Decimal(ln[0]), 'lon': decimal.Decimal(lt[0]), 'label': label}, ignore_index=True)
            elif len(ln[1])==1 and (ln[1] == 'E' or ln[1] == 'W'):
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': label}, ignore_index=True)
            elif (lt[1] == 'E' or lt[1] == 'W') and len(ln[1])>1:
                coords = coords.append({'lat': decimal.Decimal(ln[0]), 'lon': decimal.Decimal(lt[0]), 'label': label}, ignore_index=True)
            elif (lt[1] == 'N' or lt[1] == 'S') and len(ln[1])>1:
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': label}, ignore_index=True)
            else:
                coords = coords.append({'lat': decimal.Decimal(lt[0]), 'lon': decimal.Decimal(ln[0]), 'label': label}, ignore_index=True)
        

# Now we have all valid coordinates. Define latitude, longitude and label from prevalid lines


In [17]:
print('Unable to process:')
for l in invalidLines:
    print(l)
# print(validLines)
# print(preValidLines)
# print(invalidLines)
# print(coords)

Unable to process:
,31.9686, 99.9018
31.9686, 99.9018,
31.96,86, 99.9018
31.9686 99.9018
this is a test
invalid, coordinates
hi there, how are you
31.N9686 , 99.W9018
91.21, 75.22
23.55, 182.33
-91.21, 75.22
23.55, -182.33
-190.22, 58.332
-34.0489 N, -111.0937 W
11.0937 W, 94.0489 N
94.0489 S, 11.0937 W
34.0489 N, -111.0937 W Any Area
3.22121212, 5.25454545454
3.22, 5.25454545454


In [18]:
print(coords) # these are final coordinates

       lat       lon                  label
0  31.9686   99.9018                  Texas
1  36.7783  119.4179            California 
2  34.0489  111.0937                       
3  34.4452  111.5584                       
4  36.1716  115.1391                       
5     45.9     170.5               Dunedin 
6   -50.00      23.6                       
7     23.6     50.00       This is a Label 
8  50.5556   23.6999  This is also a Label 


In [19]:
# Prepare geojson string
geojson_string='{"type": "FeatureCollection", "features": ['
for index, row in coords.iterrows():
    
    lt="{}".format(row['lat'])
    ln="{}".format(row['lon'])
    
    geojson_string += '{"type": "Feature","properties": {"label": "'+ row['label'] +'"},'
    geojson_string += '"geometry": {"type": "Point","coordinates": ['+ ln +','+ lt +']}}'
    
    if index != len(coords)-1:
        geojson_string += ','
    
#     geojson_string += "{'type': 'Feature', 'properties': { 'label': '"+ row['label'] +"' },"
#     geojson_string += " 'geometry': { 'type': 'Point', 'coordinates': ["+ ln +", "+ lt +"]}},"
    
geojson_string += "]}"

print(geojson_string)    

{"type": "FeatureCollection", "features": [{"type": "Feature","properties": {"label": "Texas"},"geometry": {"type": "Point","coordinates": [99.9018,31.9686]}},{"type": "Feature","properties": {"label": "California "},"geometry": {"type": "Point","coordinates": [119.4179,36.7783]}},{"type": "Feature","properties": {"label": ""},"geometry": {"type": "Point","coordinates": [111.0937,34.0489]}},{"type": "Feature","properties": {"label": ""},"geometry": {"type": "Point","coordinates": [111.5584,34.4452]}},{"type": "Feature","properties": {"label": ""},"geometry": {"type": "Point","coordinates": [115.1391,36.1716]}},{"type": "Feature","properties": {"label": "Dunedin "},"geometry": {"type": "Point","coordinates": [170.5,45.9]}},{"type": "Feature","properties": {"label": ""},"geometry": {"type": "Point","coordinates": [23.6,-50.00]}},{"type": "Feature","properties": {"label": "This is a Label "},"geometry": {"type": "Point","coordinates": [50.00,23.6]}},{"type": "Feature","properties": {"labe

In [20]:
# write geojson string to geojson file
if os.path.exists("points.json"):
    os.remove("points.json")
f = open("points.json", "w")
f.write(geojson_string)
f.close()

In [23]:
# read geojson from json file and add them to map as marker/point
points = f"points.json"

m = folium.Map(
    location=[-59.1759, -11.6016],
    zoom_start=1,
)

folium.GeoJson(points, name="Points").add_to(m)

folium.LayerControl().add_to(m)


In [24]:
m